In [4]:
import requests
headers = {
 'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36'   
}
res = requests.get('https://sale.591.com.tw/home/search/list?type=2&&shType=list&section=7&regionid=1&kind=9&firstRow=30&totalRows=2404&timestamp=1539754751297', headers = headers)

In [5]:
res

<Response [200]>

In [11]:
import pandas
pandas.DataFrame(res.json()['data']['house_list']).head(3)

,address,area,browsenum,cartmodel,carttype,community_link,community_name,delivery,down_price_percent,fci_pai,...,saletype,section_name,shape_name,showhouseage,showprice,tag,title,type,unit_price,unitprice
0,忠孝東路五段31巷,15.16,382,,,/newCommunity-index.html?cid=50519,無爭,,NaN,,...,0,信義區,電梯大樓,10年,"1,790",[],市府極美無爭精品,2,118.07萬/坪,118.07
1,忠孝東路五段743巷,32.56,606,,,/newCommunity-index.html?cid=93550,達麗信義,,NaN,,...,0,信義區,電梯大樓,3年,"2,600","[含車位, 有陽台]",指名承辦人小鍾：本棟最低總價！精緻裝潢！,2,114.77萬/坪,114.77
2,松山路,20.68,720,,,/newCommunity-index.html?cid=6751,永春大樓,,NaN,,...,0,信義區,電梯大樓,34年,"1,220",[有陽台],專售永春大樓-住商住匠團隊,2,58.99萬/坪,58.99


In [14]:
import time
houseurl = 'https://sale.591.com.tw/home/search/list?type=2&&shType=list&section=7&regionid=1&kind=9&firstRow={}&totalRows=2404&timestamp=1539754751297'

houseary = []
for i in range(0,int(2404 / 30)+ 1 ):
    res = requests.get(houseurl.format(i*30), headers = headers)
    df = pandas.DataFrame(res.json()['data']['house_list'])
    houseary.append(df)
    time.sleep(1)
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80


In [15]:
housedf = pandas.concat(houseary)

In [17]:
housedf.count()

address                 2403
area                    2405
browsenum               2405
build_purpose              2
call_num                   2
cartmodel               2403
carttype                2403
community_link          2403
community_name          2403
company                    2
delivery                2403
down_price_percent        35
fci_pai                 2403
floor                   2403
has_carport             2403
houseage                2405
houseid                 2405
is_carport              2403
is_combine              2403
is_down_price           2403
is_full                    2
is_hurry_price          2403
is_newhouse                2
is_oversea              2403
is_pro_advertisement       2
is_video                2403
isnew                   2403
isvip                   2403
kind                    2403
kind_name               2403
label                      2
mainarea                2403
nick_name               2403
original_price            35
phonenum      

In [18]:
housedf.to_excel('house591.xlsx')